In [1]:
import json
import numpy as np
import pandas as pd
import random
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pickle
from datetime import timedelta
from datetime import datetime

In [2]:
activity = pd.read_csv('/scratch/stoll/BiGAN/data/ibat/initial/raw_results_demo.csv', header=0).fillna(-200)
activity.head()

,date_format,epoch_format,zigduino-3:humidity,zigduino-3:temperature,zigduino-4:humidity,zigduino-4:temperature
0,2020-01-15 00:00:00,1.579046e+12,10.667487,20.460420,1.0,22.143732
1,2020-01-15 02:00:00,1.579054e+12,10.670172,20.334833,1.0,22.045699
2,2020-01-15 04:00:00,1.579061e+12,10.368995,20.527430,1.0,22.896719
3,2020-01-15 06:00:00,1.579068e+12,10.147413,20.864872,1.0,23.479302
4,2020-01-15 08:00:00,1.579075e+12,9.357395,21.624195,1.0,23.749600


In [3]:
activity["timestamp"] = pd.to_datetime(activity["date_format"])
activity["Date"] = pd.to_datetime(activity["timestamp"].dt.date, utc=False)
# activity["Time"] = activity["timestamp"].dt.time
activity["Time"] = activity["timestamp"].dt.time.astype(str)
activity["Month"] = activity.Date.dt.month


activity.dtypes

date_format                       object
epoch_format                     float64
zigduino-3:humidity              float64
zigduino-3:temperature           float64
zigduino-4:humidity              float64
zigduino-4:temperature           float64
timestamp                 datetime64[ns]
Date                      datetime64[ns]
Time                              object
Month                              int64
dtype: object

In [4]:
activity.head()

,date_format,epoch_format,zigduino-3:humidity,zigduino-3:temperature,zigduino-4:humidity,zigduino-4:temperature,timestamp,Date,Time,Month
0,2020-01-15 00:00:00,1.579046e+12,10.667487,20.460420,1.0,22.143732,2020-01-15 00:00:00,2020-01-15,00:00:00,1
1,2020-01-15 02:00:00,1.579054e+12,10.670172,20.334833,1.0,22.045699,2020-01-15 02:00:00,2020-01-15,02:00:00,1
2,2020-01-15 04:00:00,1.579061e+12,10.368995,20.527430,1.0,22.896719,2020-01-15 04:00:00,2020-01-15,04:00:00,1
3,2020-01-15 06:00:00,1.579068e+12,10.147413,20.864872,1.0,23.479302,2020-01-15 06:00:00,2020-01-15,06:00:00,1
4,2020-01-15 08:00:00,1.579075e+12,9.357395,21.624195,1.0,23.749600,2020-01-15 08:00:00,2020-01-15,08:00:00,1


In [5]:
cols = activity.columns.to_list()
cols = cols[-3:] + cols[:-3]
activity = activity[cols]
# activity.drop(["date_format", "timestamp"], axis=1, inplace=True)


In [6]:
activity.isna().sum()

Date                      0
Time                      0
Month                     0
date_format               0
epoch_format              0
zigduino-3:humidity       0
zigduino-3:temperature    0
zigduino-4:humidity       0
zigduino-4:temperature    0
timestamp                 0
dtype: int64

In [7]:
rows = activity.groupby('Month').count()['Date']%20
rows

Month
1     4
2     8
3    12
4    10
Name: Date, dtype: int64

In [8]:
rows = pd.DataFrame(rows)
rows = rows.reset_index()

In [9]:
final = pd.DataFrame()
for seq in range(rows.shape[0]):
    #print(rows.iloc[seq,0])
    #print(rows.iloc[seq,1])
    
    temp = activity[activity['Month'] == rows.iloc[seq, 0]]
    #print(temp.shape)
    nrows = temp.shape[0] - rows.iloc[seq, 1]
    temp = temp.iloc[0:nrows, :]
    #print(temp.shape)
    #print(temp.shape[0]%20)
    final = pd.concat([final, temp])

final.shape

(1220, 10)

In [10]:
final.groupby('Month').count()['Date']%20

Month
1    0
2    0
3    0
4    0
Name: Date, dtype: int64

In [11]:
final.head()

,Date,Time,Month,date_format,epoch_format,zigduino-3:humidity,zigduino-3:temperature,zigduino-4:humidity,zigduino-4:temperature,timestamp
0,2020-01-15,00:00:00,1,2020-01-15 00:00:00,1.579046e+12,10.667487,20.460420,1.0,22.143732,2020-01-15 00:00:00
1,2020-01-15,02:00:00,1,2020-01-15 02:00:00,1.579054e+12,10.670172,20.334833,1.0,22.045699,2020-01-15 02:00:00
2,2020-01-15,04:00:00,1,2020-01-15 04:00:00,1.579061e+12,10.368995,20.527430,1.0,22.896719,2020-01-15 04:00:00
3,2020-01-15,06:00:00,1,2020-01-15 06:00:00,1.579068e+12,10.147413,20.864872,1.0,23.479302,2020-01-15 06:00:00
4,2020-01-15,08:00:00,1,2020-01-15 08:00:00,1.579075e+12,9.357395,21.624195,1.0,23.749600,2020-01-15 08:00:00


In [12]:
final.groupby('Month').count()['Time']

Month
1    200
2    340
3    360
4    320
Name: Time, dtype: int64

In [13]:
(final[final['zigduino-3:temperature'] == -200].count())

Date                      82
Time                      82
Month                     82
date_format               82
epoch_format              82
zigduino-3:humidity       82
zigduino-3:temperature    82
zigduino-4:humidity       82
zigduino-4:temperature    82
timestamp                 82
dtype: int64

In [14]:
# activityTrain = final[final['Month'].isin([3, 4, 6, 8, 9, 11, 12, 13, 15, 16])]
# activityVal = final[final['Month'].isin([5])]
# activityTest = final[final['Month'].isin([7, 10, 14])]

activityTrain = final[final['Month'].isin([1, 4])]
activityVal = final[final['Month'].isin([3])]
activityTest = final[final['Month'].isin([2])]

In [15]:
# maskTrain = pd.DataFrame(np.ones((activityTrain.shape[0], activityTrain.shape[1])), columns=activityTrain.columns)
# maskTrain['Time']=activityTrain['Time']

In [16]:
maskTrain = activityTrain[['zigduino-3:temperature', 'Date', 'Time']].copy()
maskTrain[maskTrain['zigduino-3:temperature'] != -200] = 1
maskTrain[maskTrain['zigduino-3:temperature'] == -200] = 0
maskTrain[['Date','Time']] = activityTrain[['Date', 'Time']].copy()

maskVal=activityVal[['zigduino-3:temperature', 'Date', 'Time']].copy()
maskVal[maskVal['zigduino-3:temperature'] != -200] = 1
maskVal[maskVal['zigduino-3:temperature'] == -200] = 0
maskVal[['Date', 'Time']] = activityVal[['Date', 'Time']].copy()

maskTest=activityTest[['zigduino-3:temperature', 'Date', 'Time']].copy()
maskTest[maskTest['zigduino-3:temperature'] != -200] = 1
maskTest[maskTest['zigduino-3:temperature'] == -200] = 0
maskTest[['Date', 'Time']] = activityTest[['Date', 'Time']].copy()


In [17]:
print(activityTest['zigduino-3:temperature'].isna().sum())

0


In [18]:
activityTest = activityTest.fillna(0)

In [19]:
maskTrain.head()

,zigduino-3:temperature,Date,Time
0,1.0,2020-01-15,00:00:00
1,1.0,2020-01-15,02:00:00
2,1.0,2020-01-15,04:00:00
3,1.0,2020-01-15,06:00:00
4,1.0,2020-01-15,08:00:00


In [30]:
def rdecay(data):
    print(data.head())
    data['intervalReverse'] = 0
    j = data.shape[0] - 1
    for n in range(int(data.shape[0]/20)):
        i = 0
        df_group = data.iloc[n * 20:(n * 20) + 20, :]
        df_group = df_group[::-1]
        for index, row in df_group.iterrows(): #go over mask
            if(i==0):
                row['intervalReverse'] = 0
                i=1
            else:
                if(prev['zigduino-3:temperature'] == 1):
                    row['intervalReverse'] = timedelta.total_seconds(datetime.strptime(str(row['Date'])[:10] + " " + str(row['Time']), "%Y-%m-%d %H:%M:%S")
                                                                   - datetime.strptime(str(prev['Date'])[:10] + " " + str(prev['Time']), "%Y-%m-%d %H:%M:%S"))
                elif(prev['zigduino-3:temperature'] == 0):
                    row['intervalReverse'] = timedelta.total_seconds(datetime.strptime(str(row['Date'])[:10] + " " + str(row['Time']), "%Y-%m-%d %H:%M:%S")
                                                                   - datetime.strptime(str(prev['Date'])[:10] + " " + str(prev['Time']), "%Y-%m-%d %H:%M:%S")) + prev['interval']
            prev = row
            data.iloc[j,4] = row['intervalReverse']
            j = j - 1

    data['intervalReverse'] = data['intervalReverse'].apply(lambda x: abs(x/60))
    print(data.head())
    return data

In [31]:
#decay
def decay(data):
    print(data.head())
    data['interval'] = 0
    # df=data.groupby('person_id')
    j=0
    for n in range(int(data.shape[0]/20)):
        i=0
        # print(n)
        df_group = data.iloc[n * 20:(n * 20) + 20, :]
        for index, row in df_group.iterrows(): # go over mask
            try:
                if(i==0):
                    row['interval']=0
                    i=1
                else:
                    if(prev['zigduino-3:temperature'] == 1):
                        row['interval'] = timedelta.total_seconds(datetime.strptime(str(row['Date'])[:10] + " " + str(row['Time']), "%Y-%m-%d %H:%M:%S")
                                                                - datetime.strptime(str(prev['Date'])[:10] + " " + str(prev['Time']), "%Y-%m-%d %H:%M:%S"))
                    elif(prev['zigduino-3:temperature'] == 0):
                        row['interval'] = timedelta.total_seconds(datetime.strptime(str(row['Date'])[:10] + " " + str(row['Time']), "%Y-%m-%d %H:%M:%S")
                                                                - datetime.strptime(str(prev['Date'])[:10] + " " + str(prev['Time']), "%Y-%m-%d %H:%M:%S")) + prev['interval']
            except ValueError as e:
                print(e)
                print(str(row['Date']) + " " + str(row['Time']))
                break

            prev = row
            data.iloc[j, 3] = row['interval']
            j = j + 1

    data['interval'] = data['interval'].apply(lambda x: abs(x/60))
    print(data.head())
    return data

maskTrain = decay(maskTrain)
print(maskTrain.head())

   zigduino-3:temperature       Date      Time  interval
0                     1.0 2020-01-15  00:00:00       0.0
1                     1.0 2020-01-15  02:00:00     120.0
2                     1.0 2020-01-15  04:00:00     120.0
3                     1.0 2020-01-15  06:00:00     120.0
4                     1.0 2020-01-15  08:00:00     120.0
   zigduino-3:temperature       Date      Time  interval
0                     1.0 2020-01-15  00:00:00       0.0
1                     1.0 2020-01-15  02:00:00     120.0
2                     1.0 2020-01-15  04:00:00     120.0
3                     1.0 2020-01-15  06:00:00     120.0
4                     1.0 2020-01-15  08:00:00     120.0
   zigduino-3:temperature       Date      Time  interval
0                     1.0 2020-01-15  00:00:00       0.0
1                     1.0 2020-01-15  02:00:00     120.0
2                     1.0 2020-01-15  04:00:00     120.0
3                     1.0 2020-01-15  06:00:00     120.0
4                     1.0 2020-

In [32]:
maskTrain = decay(maskTrain)
print(maskTrain.head())
maskTrain = rdecay(maskTrain)

   zigduino-3:temperature       Date      Time  interval
0                     1.0 2020-01-15  00:00:00       0.0
1                     1.0 2020-01-15  02:00:00     120.0
2                     1.0 2020-01-15  04:00:00     120.0
3                     1.0 2020-01-15  06:00:00     120.0
4                     1.0 2020-01-15  08:00:00     120.0
   zigduino-3:temperature       Date      Time  interval
0                     1.0 2020-01-15  00:00:00       0.0
1                     1.0 2020-01-15  02:00:00     120.0
2                     1.0 2020-01-15  04:00:00     120.0
3                     1.0 2020-01-15  06:00:00     120.0
4                     1.0 2020-01-15  08:00:00     120.0
   zigduino-3:temperature       Date      Time  interval
0                     1.0 2020-01-15  00:00:00       0.0
1                     1.0 2020-01-15  02:00:00     120.0
2                     1.0 2020-01-15  04:00:00     120.0
3                     1.0 2020-01-15  06:00:00     120.0
4                     1.0 2020-

In [33]:
maskVal = decay(maskVal)
print(maskVal.head())
maskVal = rdecay(maskVal)

     zigduino-3:temperature       Date      Time
552                     1.0 2020-03-01  00:00:00
553                     1.0 2020-03-01  02:00:00
554                     1.0 2020-03-01  04:00:00
555                     1.0 2020-03-01  06:00:00
556                     1.0 2020-03-01  08:00:00
     zigduino-3:temperature       Date      Time  interval
552                     1.0 2020-03-01  00:00:00       0.0
553                     1.0 2020-03-01  02:00:00     120.0
554                     1.0 2020-03-01  04:00:00     120.0
555                     1.0 2020-03-01  06:00:00     120.0
556                     1.0 2020-03-01  08:00:00     120.0
     zigduino-3:temperature       Date      Time  interval
552                     1.0 2020-03-01  00:00:00       0.0
553                     1.0 2020-03-01  02:00:00     120.0
554                     1.0 2020-03-01  04:00:00     120.0
555                     1.0 2020-03-01  06:00:00     120.0
556                     1.0 2020-03-01  08:00:00     120.

In [34]:
maskTest = decay(maskTest)
print(maskTest.head())
maskTest = rdecay(maskTest)

     zigduino-3:temperature       Date      Time
204                     0.0 2020-02-01  00:00:00
205                     0.0 2020-02-01  02:00:00
206                     0.0 2020-02-01  04:00:00
207                     0.0 2020-02-01  06:00:00
208                     0.0 2020-02-01  08:00:00
     zigduino-3:temperature       Date      Time  interval
204                     0.0 2020-02-01  00:00:00       0.0
205                     0.0 2020-02-01  02:00:00     120.0
206                     0.0 2020-02-01  04:00:00     240.0
207                     0.0 2020-02-01  06:00:00     360.0
208                     0.0 2020-02-01  08:00:00     480.0
     zigduino-3:temperature       Date      Time  interval
204                     0.0 2020-02-01  00:00:00       0.0
205                     0.0 2020-02-01  02:00:00     120.0
206                     0.0 2020-02-01  04:00:00     240.0
207                     0.0 2020-02-01  06:00:00     360.0
208                     0.0 2020-02-01  08:00:00     480.

In [36]:
activityTrain.to_csv('/scratch/stoll/BiGAN/data/ibat/preprocess/airTrain.csv', index=False)
maskTrain.to_csv('/scratch/stoll/BiGAN/data/ibat/preprocess/airTrainMask.csv', index=False)

In [150]:
# activityVal.to_csv('.../aaai/data/mimic/preprocess/airVal.csv', index=False)
# maskVal.to_csv('.../aaai/data/mimic/preprocess/airValMask.csv', index=False)

In [151]:
# activityTest.to_csv('.../aaai/data/mimic/preprocess/airTest.csv', index=False)
# maskTest.to_csv('.../aaai/data/mimic/preprocess/airTestMask.csv', index=False)